<a href="https://colab.research.google.com/github/humazafar2703/Assignment3_Python/blob/main/Seq2SeqLSTM_rainfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Assuming the uploaded file is the correct CSV, load it into a DataFrame
  try:
    data = pd.read_csv(fn)
    print("\nFile loaded successfully. Here's the head of the DataFrame:")
    print(data.head())
  except Exception as e:
    print(f"\nError loading file {fn}: {e}")

Saving all_weather_data.csv to all_weather_data.csv
User uploaded file "all_weather_data.csv" with length 169272479 bytes

File loaded successfully. Here's the head of the DataFrame:
      location        date  min_temp °c  max_temp °c  rain mm  humidity %  \
0     Holywood  2009-01-01          0.0          3.0      0.0        86.0   
1   North Cray  2009-01-01         -3.0          2.0      0.0        93.0   
2  Portknockie  2009-01-01          2.0          4.0      0.8        88.0   
3  Blairskaith  2009-01-01         -3.0          1.0      0.0        86.0   
4     Onehouse  2009-01-01         -1.0          3.0      0.0        91.0   

   cloud_cover %  wind_speed km/h wind_direction  wind_direction_numerical  
0           14.0             12.0              E                      90.0  
1           44.0              8.0            NNE                      22.5  
2           87.0             10.0            ESE                     112.5  
3           43.0             12.0            E

In [21]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Scale the 'rain mm' and 'location_encoded' columns
data_selected[['rain_scaled', 'location_encoded_scaled']] = scaler.fit_transform(data_selected[['rain mm', 'location_encoded']])

# Display the head of the DataFrame with the new scaled columns
print("DataFrame with scaled features:")
print(data_selected.head())

DataFrame with scaled features:
    location        date  min_temp °c  max_temp °c  rain mm  humidity %  \
0     Madrid  2009-01-01          4.0         10.0      0.5        91.0   
1   Bearsden  2009-01-01         -3.0          1.0      0.0        86.0   
2  Waterside  2009-01-01         -2.0          2.0      0.0        81.0   
3     Athens  2009-01-01          3.0         11.0      0.0        75.0   
4   Bridgend  2009-01-01         -1.0          3.0      0.0        82.0   

   cloud_cover %  wind_speed km/h wind_direction  wind_direction_numerical  \
0           65.0              4.0            SSW                     202.5   
1           43.0             12.0            ENE                      67.5   
2           33.0             16.0              E                      90.0   
3           12.0             10.0            WNW                     292.5   
4           40.0             15.0            ENE                      67.5   

   location_encoded  rain_scaled  location_encod

In [22]:
# Display column names and data types
print("Column names and data types:")
print(data.info())

# Check for missing values
print("\nMissing values per column:")
print(data.isnull().sum())

# Get summary statistics for numerical columns
print("\nSummary statistics for numerical columns:")
print(data.describe())

Column names and data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2836186 entries, 0 to 2836185
Data columns (total 10 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   location                  object 
 1   date                      object 
 2   min_temp °c               float64
 3   max_temp °c               float64
 4   rain mm                   float64
 5   humidity %                float64
 6   cloud_cover %             float64
 7   wind_speed km/h           float64
 8   wind_direction            object 
 9   wind_direction_numerical  float64
dtypes: float64(7), object(3)
memory usage: 216.4+ MB
None

Missing values per column:
location                    0
date                        0
min_temp °c                 0
max_temp °c                 0
rain mm                     0
humidity %                  0
cloud_cover %               0
wind_speed km/h             0
wind_direction              0
wind_direction_numerical    0
dty

*Seq2Seq* LSTM Encoder-Decoder

In [25]:
def create_sequences(X, y, seq_length=7):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

seq_length = 7

X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

print(X_train_seq.shape, y_train_seq.shape)


(8717, 7, 7, 6) (8717,)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam

def build_seq2seq(input_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape),
        RepeatVector(input_shape[0]), # Repeat the encoded vector input_shape[0] times (sequence length)
        LSTM(64, activation='relu', return_sequences=True),
        TimeDistributed(Dense(input_shape[1])) # Output a vector of size input_shape[1] (num_features) for each step in the output sequence
    ])
    model.compile(optimizer=Adam(0.001), loss='mse')
    return model

# Use X_train (already in 3D sequence format) and its shape
# Train as an autoencoder (input = target)
seq2seq = build_seq2seq(X_train.shape[1:])
history_seq2seq = seq2seq.fit(X_train, X_train, validation_data=(X_val, X_val), epochs=50, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 0.1370 - val_loss: 0.0029
Epoch 2/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0025 - val_loss: 0.0022
Epoch 3/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0022 - val_loss: 0.0020
Epoch 4/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 5/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 6/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 7/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 8/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 9/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 10/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 11/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 12/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 

In [29]:
# prompt: evaluate the model and calculate r2 and mse

from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Assuming `model` is your trained Keras model and `X_test_scaled` is your test data
# Replace with your actual model and test data variable names
model = seq2seq  # Replace with the name of your trained model variable
X_test_for_prediction = X_test # Replace with your test data

# Make predictions
y_pred = model.predict(X_test_for_prediction)

# Flatten the predictions and actual values if they are in sequences
# This is needed to calculate metrics like R2 and MSE on the individual predicted values
# Correctly extract the prediction for the last time step in the output sequence
y_pred_last_step = y_pred[:, -1, 0] # Assuming rain_scaled is the first feature in the output sequence

# Calculate R2 score
r2 = r2_score(y_test, y_pred_last_step)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred_last_step)

print(f"R2 Score: {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
R2 Score: -64.4202
Mean Squared Error (MSE): 0.2800
